In [1]:
# Importing necessary libraries
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from utils import download_images

In [2]:
# Device configuration (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
images = pd.read_csv('../dataset/train.csv')
# images = images['image_links']
links = images['image_link']
links.head()
download_images(links, download_folder="Images")

  7%|▋         | 17896/263859 [43:16<4:10:16, 16.38it/s] 

In [ ]:
# Step 1: Data Loading and Preprocessing

# Custom Dataset for Image and Labels
class ProductDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.data = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.data.iloc[idx, 1])  # Assuming 'image_link' is the 2nd column
        image = Image.open(img_name).convert('RGB')
        label = self.data.iloc[idx, 3]  # Assuming 'entity_value' is the 4th column

        if self.transform:
            image = self.transform(image)
        
        return image, label


In [ ]:
# Data Transformation and Augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
# Load the dataset
train_df = pd.read_csv('dataset/train.csv')
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

train_dataset = ProductDataset(train_data, 'path_to_images', transform=data_transforms['train'])
val_dataset = ProductDataset(val_data, 'path_to_images', transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# Step 2: Model Definition

class ProductModel(nn.Module):
    def __init__(self, num_classes):
        super(ProductModel, self).__init__()
        self.backbone = models.resnet50(pretrained=True)  # Using ResNet50 for feature extraction
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(x)
        return x


In [ ]:
# Step 3: Training the Model

def train_model(model, criterion, optimizer, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                data_loader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                data_loader = val_loader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimization in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / len(data_loader.dataset)
            print(f'{phase} Loss: {epoch_loss:.4f}')

    print('Training complete')

In [ ]:
# Initialize model, loss function, and optimizer
num_classes = len(train_df['entity_value'].unique())  # Number of unique labels in the dataset
model = ProductModel(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, criterion, optimizer, num_epochs=10)

# Save the model after training
torch.save(model.state_dict(), 'product_model.pth')